In [45]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [46]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import pickle
from sklearn.neighbors import NearestNeighbors 
from scipy.sparse import csr_matrix
import csv

In [47]:
book = pd.read_csv('../input/bookcrossing-dataset/Book reviews/Book reviews/BX_Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")
user = pd.read_csv('../input/bookcrossing-dataset/Book reviews/Book reviews/BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
rating = pd.read_csv('../input/bookcrossing-dataset/Book reviews/Book reviews/BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")

In [48]:
book.columns = ['ISBN', 'BookTitle', 'BookAuthor', 'YearOfPublication', 'Publisher', 'ImageUrlS', 'ImageUrlM', 'ImageUrlL']
user.columns = ['UserID', 'Location', 'Age']
rating.columns = ['UserID', 'ISBN', 'BookRating'] 

In [49]:
book

In [50]:
user

In [51]:
rating

In [52]:
c1_usid = rating['UserID'].value_counts()  
c1_usid

In [53]:
rating = rating[rating['UserID'].isin(c1_usid[c1_usid >= 10].index)]
rating['UserID'].value_counts()

In [54]:
c2_usid = rating['BookRating'].value_counts()  
c2_usid

In [55]:
rating = rating[rating['BookRating'].isin(c2_usid[c2_usid >= 10].index)]
c1 = rating['BookRating'].value_counts()
c1

In [56]:
book_rating = pd.merge(rating, book, on='ISBN')
book_rating

In [57]:
columns = ['BookAuthor', 'YearOfPublication', 'Publisher', 'ImageUrlS', 'ImageUrlM', 'ImageUrlL']
book_rating = book_rating.drop(columns, axis=1)
book_rating

In [58]:
book_rating = book_rating.dropna()
book_rating

In [59]:
bkrating_cnt = (book_rating.groupby(by = ['BookTitle'])['BookRating'].count().reset_index().rename(
columns = {'BookRating': 'TotalRating'})[['BookTitle', 'TotalRating']]) 
bkrating_cnt

In [60]:
rating_bkrating_cnt = book_rating.merge(bkrating_cnt, left_on = 'BookTitle', right_on = 'BookTitle', how = 'left')
rating_bkrating_cnt

In [61]:
a = rating_bkrating_cnt['TotalRating']
a[a>=10].value_counts()

In [62]:
a[a>=10]

In [63]:
b=rating_bkrating_cnt[rating_bkrating_cnt['TotalRating']>=10]
b

In [64]:
user.isna().sum()

In [65]:
user['Age'].unique()

In [66]:
user['Age'] = user['Age'].fillna(user['Age'].mode()[0])

In [67]:
from sklearn.preprocessing import LabelEncoder  
label_encoder_y= LabelEncoder() 

user['Location']= label_encoder_y.fit_transform(user['Location'])

In [68]:
corrMatrix = user.corr()
plt.figure(figsize = (15,15))
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [69]:
user.drop('Age',axis=1,inplace=True)
user.drop('Location',axis=1,inplace=True)
user

In [70]:
final = b.merge(user,left_on = 'UserID', right_on = 'UserID', how = 'left')
final

In [71]:
final = final.drop_duplicates(['UserID', 'BookTitle'])
final

In [72]:
final.to_csv('Final.csv')

In [73]:
final_pivot = final.pivot(index = 'BookTitle', columns = 'UserID', values = 'BookRating').fillna(0)
final_matrix = csr_matrix(final_pivot.values)

In [74]:
model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model.fit(final_matrix)

In [75]:
finalmodel = 'Final_Model.sav'
pickle.dump(model, open(finalmodel, 'wb'))

In [76]:
final_pivot

In [77]:
final_matrix

In [78]:
final_pivot.shape[0]

In [79]:
distances, suggestions = model.kneighbors(final_pivot.iloc[237, :].values.reshape(1, -1),n_neighbors = 5)

In [80]:
for i in range(len(suggestions)):
    print(final_pivot.index[suggestions[i]])

In [81]:
nindex = np.random.choice(final_pivot.shape[0])
for i in range(0, len(distances.flatten())):
        if i == 0:
            print('Recommendations :\n'.format(final_pivot.index[nindex]))
        else:
            print('{0}: {1}'.format(i, final_pivot.index[suggestions.flatten()[i]],distances.flatten()[i]))

In [82]:
bkname = []
for i in range(final_pivot.shape[0]):
    bkname.append(final_pivot.index[i])

In [83]:
bkname